In [1]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [11]:
def load_point_cloud(filepath, visualize=False):
    pcd = o3d.io.read_point_cloud(filepath)
    print(f"[INFO] Loaded scan with {len(pcd.points)} points.")
    if visualize:
        o3d.visualization.draw_geometries([pcd])
    return pcd

def estimate_point_density(pcd, radius_mm=1.0, min_neighbors=30):
    kdtree = o3d.geometry.KDTreeFlann(pcd)
    counts = []
    total_checked = 0
    for i in range(0, len(pcd.points), max(1, len(pcd.points)//1000)):  # sample ~1000 points
        [_, idx, _] = kdtree.search_radius_vector_3d(pcd.points[i], radius_mm)
        neighbor_count = len(idx) - 1  # exclude the point itself
        if neighbor_count >= min_neighbors:
            counts.append(neighbor_count)
        total_checked += 1

    if counts:
        avg_density = np.mean(counts)
        print(f"[INFO] Average density (excluding sparse points): {avg_density:.2f} points per {radius_mm}mm sphere")
        print(f"[INFO] Used {len(counts)} / {total_checked} sampled points (≥ {min_neighbors} neighbors)")
    else:
        avg_density = 0
        print(f"[WARN] No sampled points had ≥ {min_neighbors} neighbors.")

    return avg_density


pcd = load_point_cloud("/home/chris/Code/PointClouds/data/FLIPscans/Bendy/Bendy_1/scan1_Part1.ply", visualize=True)
avg_density = estimate_point_density(pcd, radius_mm=1.0)

[INFO] Loaded scan with 1492459 points.
[INFO] Average density (excluding sparse points): 288.62 points per 1.0mm sphere
[INFO] Used 998 / 1001 sampled points (≥ 30 neighbors)


In [30]:
def show_cad_adjusted(cad_path, z_rotation=0, x_rotation=0, y_rotation=0):
    mesh = o3d.io.read_triangle_mesh(cad_path)
    mesh.compute_vertex_normals()

    # Combine rotations in ZYX order
    Rz = mesh.get_rotation_matrix_from_axis_angle([0, 0, np.deg2rad(z_rotation)])
    Rx = mesh.get_rotation_matrix_from_axis_angle([np.deg2rad(x_rotation), 0, 0])
    Ry = mesh.get_rotation_matrix_from_axis_angle([0, np.deg2rad(y_rotation), 0])

    # Apply rotations
    mesh.rotate(Rx, center=mesh.get_center())
    mesh.rotate(Ry, center=mesh.get_center())
    mesh.rotate(Rz, center=mesh.get_center())
    mesh.paint_uniform_color([0.6, 0.6, 0.9])

    o3d.visualization.draw_geometries([mesh],
        front=[0, 0, -1],  # camera looks down
        up=[0, -1, 0]      # keeps left/right orientation intuitive
    )

# Try different values to see which aligns "top" to Z:
show_cad_adjusted("/home/chris/Code/PointClouds/data/FLIPscans/Bendy/BendyCAD.STL", x_rotation=290, y_rotation=0, z_rotation=270)

In [ ]:
def raycast_mesh(mesh, origin_offset=10, target_density=300):
    import open3d as o3d
    import numpy as np

    mesh.compute_triangle_normals()

    # Apply rotation (already rotated)
    t_mesh = o3d.t.geometry.TriangleMesh.from_legacy(mesh)
    scene = o3d.t.geometry.RaycastingScene()
    _ = scene.add_triangles(t_mesh)

    # Bounding box
    bbox = mesh.get_axis_aligned_bounding_box()
    min_x, min_y, _ = bbox.min_bound
    max_x, max_y, _ = bbox.max_bound
    width = max_x - min_x
    height = max_y - min_y

    # Estimate point spacing for target density
    spacing = (4.19 / target_density) ** (1/3)  # mm per point
    num_x = int(width / spacing)
    num_y = int(height / spacing)

    print(f"[INFO] Ray grid: {num_x} x {num_y} ({spacing:.3f} mm spacing)")

    # Grid
    x_vals = np.linspace(min_x, max_x, num_x)
    y_vals = np.linspace(min_y, max_y, num_y)
    xx, yy = np.meshgrid(x_vals, y_vals)

    # Rays
    origin_z = bbox.max_bound[2] + origin_offset
    origins = np.stack([xx.ravel(), yy.ravel(), np.full(xx.size, origin_z)], axis=1)
    directions = np.tile([0, 0, -1], (origins.shape[0], 1))

    rays = o3d.core.Tensor(np.hstack((origins, directions)), dtype=o3d.core.Dtype.Float32)
    hits = scene.cast_rays(rays)

    mask = hits['t_hit'].isfinite()
    hit_points = origins[mask.numpy()] + hits['t_hit'][mask].numpy().reshape(-1, 1) * directions[mask.numpy()]
    pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(hit_points))

    print(f"[INFO] Generated point cloud with {len(pcd.points)} points.")
    return pcd


x_rotation=290
y_rotation=0
z_rotation=270

mesh = o3d.io.read_triangle_mesh("/home/chris/Code/PointClouds/data/FLIPscans/Bendy/BendyCAD.STL")
# Apply rotation manually
# Combine rotations in ZYX order
Rz = mesh.get_rotation_matrix_from_axis_angle([0, 0, np.deg2rad(z_rotation)])
Rx = mesh.get_rotation_matrix_from_axis_angle([np.deg2rad(x_rotation), 0, 0])
Ry = mesh.get_rotation_matrix_from_axis_angle([0, np.deg2rad(y_rotation), 0])

# Apply rotations
mesh.rotate(Rx, center=mesh.get_center())
mesh.rotate(Ry, center=mesh.get_center())
mesh.rotate(Rz, center=mesh.get_center())

mesh.paint_uniform_color([0.6, 0.6, 0.9])

pcd = raycast_mesh(mesh)
o3d.visualization.draw_geometries([pcd],front=[0, 0, -1], up=[0, -1, 0])



[INFO] Ray grid: 234 x 311 (0.241 mm spacing)
[INFO] Generated point cloud with 55552 points.
